In [ ]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os


DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

table_main = 'ratehawk'


def create_sql_file(output_file_path):
    """
    Creates an SQL file that contains a query to fetch data from `innova_hotels_main` table
    where SupplierCode is "Ratehawk".

    :param output_file_path: The path where the SQL file will be saved.
    """
    sql_query = "SELECT * FROM innova_hotels_main WHERE SupplierCode=\"Ratehawk\";"

    try:
        # Write the SQL query to the file
        with open(output_file_path, "w") as sql_file:
            sql_file.write(sql_query)

        print(f"SQL file successfully created at: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while creating the SQL file: {e}")

# Define the output file path (change this to your desired path)
output_path = os.path.join(os.getcwd(), "ratehawk_data.sql")

# Create the SQL file
create_sql_file(output_path)


In [ ]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os

# Database URL for connecting to MySQL
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

def fetch_ratehawk_data():
    """
    Fetches data from the `innova_hotels_main` table where SupplierCode is "Ratehawk"
    and writes the data into a text file.
    """
    connection = engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the database
    table_main = Table('innova_hotels_main', metadata, autoload_with=engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        result = connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the database.")

        # Save result to file
        output_file_path = os.path.join(os.getcwd(), "ratehawk_data.sql")
        with open(output_file_path, "w") as output_file:
            for row in result:
                output_file.write(str(row) + "\n")

        print(f"Ratehawk data saved to {output_file_path}")

    except Exception as e:
        print(f"An error occurred while fetching data: {e}")

    finally:
        connection.close()

fetch_ratehawk_data()


In [1]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os
from dotenv import load_dotenv


load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')


# Database URL for connecting to local MySQL database
LOCAL_DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
local_engine = create_engine(LOCAL_DATABASE_URL)

# Database URL for connecting to server MySQL database
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)

def fetch_and_insert_ratehawk_data():
    
    local_connection = local_engine.connect()
    server_connection = server_engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the local database
    table_main = Table('innova_hotels_main', metadata, autoload_with=local_engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        # Fetch data from the local database
        result = local_connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the local database.")

        # Insert data into the server database
        if result:
            server_table = Table('innova_hotels_main', metadata, autoload_with=server_engine)
            insert_query = insert(server_table).values([dict(row) for row in result])
            server_connection.execute(insert_query)
            print(f"Inserted {len(result)} records into the server database.")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        local_connection.close()
        server_connection.close()

fetch_and_insert_ratehawk_data()


Fetched 1713000 records from the local database.
An error occurred: cannot convert dictionary update sequence element #0 to a sequence


In [3]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
import os
from dotenv import load_dotenv


load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')


# Database URL for connecting to local MySQL database
LOCAL_DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
local_engine = create_engine(LOCAL_DATABASE_URL)

# Database URL for connecting to server MySQL database
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)


def fetch_and_insert_ratehawk_data():
    """
    Fetches data from the `innova_hotels_main` table in the local database where SupplierCode is "Ratehawk"
    and inserts the data into the same table on the server database.
    Displays progress immediately for each chunk inserted.
    """
    local_connection = local_engine.connect()
    server_connection = server_engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the local database
    table_main = Table('innova_hotels_main', metadata, autoload_with=local_engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        # Fetch data from the local database
        result = local_connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the local database.")

        # Insert data into the server database in chunks
        if result:
            server_table = Table('innova_hotels_main', metadata, autoload_with=server_engine)
            CHUNK_SIZE = 1000
            for i in range(0, len(result), CHUNK_SIZE):
                chunk = result[i:i + CHUNK_SIZE]
                insert_query = insert(server_table).values([dict(row._mapping) for row in chunk])
                server_connection.execute(insert_query)
                print(f"Inserted {len(chunk)} records into the server database. Total inserted so far: {i + len(chunk)}")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        local_connection.close()
        server_connection.close()

fetch_and_insert_ratehawk_data()


Fetched 1713000 records from the local database.
Inserted 1000 records into the server database. Total inserted so far: 1000
Inserted 1000 records into the server database. Total inserted so far: 2000
Inserted 1000 records into the server database. Total inserted so far: 3000
Inserted 1000 records into the server database. Total inserted so far: 4000
Inserted 1000 records into the server database. Total inserted so far: 5000
Inserted 1000 records into the server database. Total inserted so far: 6000
Inserted 1000 records into the server database. Total inserted so far: 7000
Inserted 1000 records into the server database. Total inserted so far: 8000
Inserted 1000 records into the server database. Total inserted so far: 9000
Inserted 1000 records into the server database. Total inserted so far: 10000
Inserted 1000 records into the server database. Total inserted so far: 11000
Inserted 1000 records into the server database. Total inserted so far: 12000
Inserted 1000 records into the serve

KeyboardInterrupt: 

In [ ]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
from sqlalchemy.exc import SQLAlchemyError
import os
from dotenv import load_dotenv


load_dotenv()

# Database connection details
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')


# Database URL for connecting to local MySQL database
LOCAL_DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
local_engine = create_engine(LOCAL_DATABASE_URL)

# Database URL for connecting to server MySQL database
SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)


def fetch_and_insert_ratehawk_data():
    """
    Fetches data from the `innova_hotels_main` table in the local database where SupplierCode is "Ratehawk"
    and inserts the data into the same table on the server database.
    Each chunk is committed immediately after insertion to ensure data persistence.
    """
    local_connection = local_engine.connect()
    server_connection = server_engine.connect()
    metadata = MetaData()

    # Reflect the table structure from the local database
    table_main = Table('innova_hotels_main', metadata, autoload_with=local_engine)

    # Create a SELECT query to fetch data
    query = select(table_main).where(table_main.c.SupplierCode == "Ratehawk")

    try:
        # Fetch data from the local database
        result = local_connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the local database.")

        # Insert data into the server database in chunks
        if result:
            server_table = Table('innova_hotels_main', metadata, autoload_with=server_engine)
            CHUNK_SIZE = 1000
            total_inserted = 0

            for i in range(0, len(result), CHUNK_SIZE):
                chunk = result[i:i + CHUNK_SIZE]

                # Use a transaction for each chunk
                with server_engine.begin() as transaction:
                    try:
                        insert_query = insert(server_table).values([dict(row._mapping) for row in chunk])
                        transaction.execute(insert_query)
                        total_inserted += len(chunk)
                        print(f"Inserted {len(chunk)} records into the server database. Total inserted so far: {total_inserted}")
                    except SQLAlchemyError as e:
                        print(f"An error occurred while inserting chunk: {e}")

        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        local_connection.close()
        server_connection.close()

fetch_and_insert_ratehawk_data()


Fetched 1713000 records from the local database.
Inserted 1000 records into the server database. Total inserted so far: 1000
Inserted 1000 records into the server database. Total inserted so far: 2000
Inserted 1000 records into the server database. Total inserted so far: 3000
Inserted 1000 records into the server database. Total inserted so far: 4000
Inserted 1000 records into the server database. Total inserted so far: 5000
Inserted 1000 records into the server database. Total inserted so far: 6000
Inserted 1000 records into the server database. Total inserted so far: 7000
Inserted 1000 records into the server database. Total inserted so far: 8000
Inserted 1000 records into the server database. Total inserted so far: 9000
Inserted 1000 records into the server database. Total inserted so far: 10000
Inserted 1000 records into the server database. Total inserted so far: 11000
Inserted 1000 records into the server database. Total inserted so far: 12000
Inserted 1000 records into the serve